In [1]:
import pandas as pd

In [2]:
info = pd.read_csv("data/inverse/info.csv", parse_dates = ["date"])
info.shape

(27756, 9)

In [3]:
ans = pd.read_csv("data/submit/1225_071_096.csv")
# ans = pd.read_csv("data/submit/submission.csv")
ans.shape

(3850, 2)

In [4]:
cust_id = {id_: i for i, id_ in enumerate(info["cust_id"].drop_duplicates().to_list())}
info["cust_id"] = info["cust_id"].map(cust_id)

In [5]:
ans_all = pd.merge(info[["alert_key", "sar_flag", "cust_id", "sample"]], ans, on = "alert_key", how = "left")

In [6]:
public = ans_all[~ans_all["probability"].isnull()].query("sample == 'train'").reset_index(drop = True)
private = ans_all[~ans_all["probability"].isnull()].query("sample == 'test'").reset_index(drop = True)
public.shape, private.shape

((1845, 5), (2005, 5))

In [7]:
target = "sar_flag"
test_sar = int(public[target].sum())
test_len = public[public[target] == 1].index[-2] + 1
print(f"Test total: {len(public)}")
print(f"({test_sar} - 1) / {test_len} = {(test_sar - 1) / test_len}")

Test total: 1845
(11 - 1) / 1594 = 0.006273525721455458


In [8]:
# train & public 的 id 重複洗錢次數
public_id1 = ans_all.query("sar_flag == 1").value_counts("cust_id")
public_id1[public_id1 != 1].head()

cust_id
7661    2
2353    2
86      2
dtype: int64

In [9]:
num = 3
public_id = public["cust_id"].to_list()
public_id_ct = public["cust_id"].value_counts()
public_id0 = public_id_ct[public_id_ct > num].index
public_id0

Int64Index([6083, 6885, 6554, 6338, 6542, 3848, 5943, 3645, 4844, 7329,
            ...
            7311, 7536,  233, 7285, 7217, 7297,  334, 7301, 5861,  298],
           dtype='int64', length=137)

In [10]:
public_idx0 = []
for i in public_id0:
    public1 = public.query("cust_id == @i")
    public1 = public1.iloc[:-num]
    # public_idx0.extend(public1["alert_key"].to_list())
    public_idx0.extend(public1.index)
len(public_idx0)

638

In [11]:
# a = public.query("alert_key in @public_idx0")
a = public.loc[public_idx0]
a.query("sar_flag == 1")

,alert_key,sar_flag,cust_id,sample,probability


In [12]:
public.loc[public_idx0, "probability"] = 0
public = public.sort_values("probability", ascending = False)

In [13]:
target = "sar_flag"
test_sar = int(public[target].sum())
test_len = public[public[target] == 1].index[-2] + 1
print(f"Test total: {len(public)}")
print(f"({test_sar} - 1) / {test_len} = {(test_sar - 1) / test_len}")

Test total: 1845
(11 - 1) / 352 = 0.028409090909090908


In [14]:
num = 3
private_id = private["cust_id"].to_list()
private_id_ct = private["cust_id"].value_counts()
private_id0 = private_id_ct[private_id_ct > num].index
private_id0

Int64Index([3848, 1809, 6885, 6542, 7412, 7500, 6554, 1012, 5618, 7329,
            ...
            7798, 8081, 7889, 7229, 5684, 6560, 7860, 7109, 7628, 5773],
           dtype='int64', length=150)

In [15]:
private_idx0 = []
for i in private_id0:
    private1 = private.query("cust_id == @i")
    private1 = private1.iloc[:-num]
    private_idx0.extend(private1.index)
len(private_idx0)

775

In [16]:
a = private.loc[private_idx0]
a.query("sar_flag == 1")

,alert_key,sar_flag,cust_id,sample,probability


In [17]:
private.loc[private_idx0, "probability"] = 0
private = private.sort_values("probability", ascending = False)

In [18]:
res = pd.concat([public, private], ignore_index = True)
res = res[["alert_key", "probability"]]

In [19]:
# res.to_csv("data/submit/1225_071_096_adjust.csv", index = False)
# res.to_csv("data/submit/1226_3d_adjust.csv", index = False)